# 코사인 유사도
- word2vec 에서 사용하고 있는 유사도 계산 방식
- 다른 방향 : -1 / 같은 방향 : 1 / 직각 : 0

In [2]:
# 선언
import numpy as np
from numpy.linalg import norm
from numpy import dot

In [3]:
# 함수 정의
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

- p1 : 저는 사과 좋아요
- p2 : 저는 바나나 좋아요
- p3 : 저는 바나나 좋아요 저는 바나나 좋아요
> 배열 순서 : 바나나, 사과, 저는, 좋아요

In [4]:
p1 = np.array([0,1,1,1])
p2 = np.array([1,0,1,1])
p3 = np.array([2,0,2,2])
p4 = np.array([1,1,1,1])
p5 = np.array([1,0,1,1])

In [5]:
print(cos_sim(p1,p2)) # 반 불일치

0.6666666666666667


In [6]:
print(cos_sim(p1,p3)) # 반 불일치

0.6666666666666667


In [7]:
print(cos_sim(p2,p3)) # 완전 일치 (가중치가 많아도 1이 최대)

1.0000000000000002


In [8]:
print(cos_sim(p2,p4)) # 4개중 1개만 불일치

0.8660254037844387


In [9]:
print(cos_sim(p2,p5)) # 완전 일치

1.0000000000000002


# 단어 사전 데이터를 이용한 의미가 유사한 단어 찾기
- kaggle : https://www.kaggle.com/borrkk/dictionary?select=dictionary.json

In [32]:
# 데이터 다루는 용도
import pandas as pd
import json

# 단어 처리
from sklearn.feature_extraction.text import TfidfVectorizer

# 각 행과의 코사인 유사도 계산 
from sklearn.metrics.pairwise import linear_kernel

In [3]:
# 파일 읽기
with open('dictionary.json', 'r') as fr:
    json_data = json.load(fr)

df = pd.DataFrame(data=[json_data.keys(), json_data.values()]).T
df.columns=['word','mean']

# 메모리 부족하다고 나와서.. 만개만 사용
df = df.head(10000)
df.head(10)

,word,mean
0,abandoned industrial site,"[Site that cannot be used for any purpose, bei..."
1,abandoned vehicle,[A vehicle that has been discarded in the envi...
2,abiotic factor,"[Physical, chemical and other non-living envir..."
3,access road,[Any street or narrow stretch of paved surface...
4,access to the sea,[The ability to bring goods to and from a port...
5,accident,"[An unexpected, unfortunate mishap, failure or..."
6,accumulator,[A rechargeable device for storing electrical ...
7,acidification,[Addition of an acid to a solution until the p...
8,acidity,[The state of being acid that is of being capa...
9,acidity degree,"[The amount of acid present in a solution, oft..."


In [5]:
# 문자 제거
for i in range(df.shape[0]):
    df['mean'][i] = str(df['mean'][i]).replace('[','')
    df['mean'][i] = str(df['mean'][i]).replace(']','')

In [7]:
df.shape

(10000, 2)

In [9]:
# null 값 확인
df['mean'].isnull().sum()

# null 이 있다면 제거
#data['mean'] = data['mean'].fillna('')

0

In [11]:
# 백터 라이저 정의(단어 학습 용도)
# - stop_words='english' 불용어(의미가 없는 단어나 조사) 제거
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['mean']) # 학습
print(tfidf_matrix.shape) # 10000 행에 15487 단어가 있다.

(10000, 15487)


In [15]:
# 각 행과의 코사인 유사도 계산 (1은 일치)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [18]:
# 단어별 인덱스 저장
indices = pd.Series(df.index, index=df['word'])
print(indices.head())

word
abandoned industrial site    0
abandoned vehicle            1
abiotic factor               2
access road                  3
access to the sea            4
dtype: int64


In [19]:
idx = indices['access road']
print(idx)

3


In [45]:
# 코사인 유사도 기반 비슷한 단어 추출
def get_recommendations(word, cosine_sim=cosine_sim):
    # 단어의 인덱스 추출
    idx = indices[word]

    # 단어와 다른 단어와의 유사값을 list로 표현 
    # [(0, 0.9999999999999998),
    # (1, 0.0),...
    # (2, 0.0)]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도가 높은 순으로 정렬 (sim_scores 의 2번째 행 기준으로 정렬 / reverse=True 내림차순)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 자기 자신을 제외한 가장 유사한 단어 10개 추출
    sim_scores = sim_scores[1:11]

    # 가장 유사한 단어의 index 추출
    word_index = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 단어 리턴
    return df['word'].iloc[word_index]

In [51]:
# 바다 유사값 : 대양, 바다 순환, 위대함, 연안, 빙산, 내륙 바다
get_recommendations('sea')

1624                 ocean
2084       sea circulation
5735             largeness
444                  coast
1159               iceberg
7648        landlocked sea
9942                marine
1244       intertidal zone
7647    landlocked country
306         brackish water
Name: word, dtype: object